In [20]:
import numpy as np
import time

# part1

In [21]:
#load data 

dictionary = np.loadtxt('data/pa5dictionary.txt',dtype='string',delimiter=' ')
training_set = np.loadtxt('data/pa5train.txt',delimiter=' ')
test_set = np.loadtxt('data/pa5test.txt',delimiter=' ')

print "dictionary size: ",dictionary.shape
print "training_set",training_set.shape
print "test_set", test_set.shape

dictionary size:  (4003,)
training_set (450, 4004)
test_set (129, 4004)


In [22]:
#predict whether this is a spam email or not,
#return 1 for spam, y for not spam
def weak_classifer(feature_in_datapoint, pos_or_neg):
    if pos_or_neg == 1:
        if feature_in_datapoint == 1:
            return 1
        else:
            return -1
    if pos_or_neg == -1:
        if feature_in_datapoint == 0:
            return 1
        else:
            return -1

In [23]:
# this boosting fucniton takes the input of data, and number of rounds to run 
# and returns a list of classifiers and a list of weighted errs which can 
# be used as the final classifier for making prediction
def boosting(dataset, round_num):
    
    weak_classifer_list = []    #tuple of feature index and (1 for hi+ and -a for hi-)
    alpha_list = []
    
    #init the weights for each data point 
    weight_list = []
    weight = 1.0/ float(dataset.shape[0])    
    for i in range (dataset.shape[0]): 
        weight_list.append(weight)

    for t in range(round_num):
       
        #-----find the best weak classifier from 4003*2 = 8006 classifiers---
        err_W_ht = 1.0
        for i in range (dataset.shape[1]-1):
            error_w_hi = 0.0    #weighted error
            
            for datapoint in range (dataset.shape[0]):   
                #hi,+
                if dataset[datapoint][i] == 1:
                    #predict spam
                    if dataset[datapoint][4003] == -1:
                        error_w_hi += weight_list[datapoint]
                if dataset[datapoint][i] == 0:
                    #predict not spam
                    if dataset[datapoint][4003] == 1:
                        error_w_hi += weight_list[datapoint]

            #hi,-  is simply calc by subtracting correct1 from num of datapoints
            error_w_hi_2 = 1.0 - error_w_hi

            #compare hi,+ and hi,- to see which is better for predicting this feature 
            if error_w_hi < error_w_hi_2:
                error = error_w_hi
                pos_or_neg  = 1
            else:
                error = error_w_hi_2
                pos_or_neg  = -1

            #update if error is reduced
            if(error < err_W_ht):
                err_W_ht = error
                final_feature = i
                final_pos_or_neg = pos_or_neg   
        
        weak_classifer_list.append((final_feature,final_pos_or_neg))
             
        #calc alpha for classifer ht from weighted err: err_W_ht
        alpha_t =  0.5 * np.log((1-err_W_ht)/err_W_ht)
        alpha_list.append(alpha_t)
        
        #calc nomalization const: Z,
        Z =  (1-err_W_ht)*np.exp(-alpha_t) + err_W_ht*np.exp(alpha_t)
        
        #update weights for all datapoints in the next run
        for datapoint in range(dataset.shape[0]):
            #if correctly classified 
            if weak_classifer(dataset[datapoint][final_feature],final_pos_or_neg) == dataset[datapoint][4003]:
                weight_list[datapoint] = weight_list[datapoint] * np.exp(-alpha_t) / Z
            else:
                weight_list[datapoint] = weight_list[datapoint] * np.exp(alpha_t) / Z
            
    return weak_classifer_list,alpha_list

In [24]:
def predict(dataset, weak_classifer_list, alpha_list):
    error_data = 0.0
    for datapoint in range(dataset.shape[0]):
        total = 0.0
        for t in range(len(weak_classifer_list)):
            total += alpha_list[t] * weak_classifer( dataset[datapoint][weak_classifer_list[t][0]], weak_classifer_list[t][1])
        if ( np.sign(total) != dataset[datapoint][4003]):
            error_data += 1
    return error_data/dataset.shape[0]

In [25]:
weak_classifer_list,alpha_list =  boosting(training_set,4)

In [27]:
print "train error after 4 rounds: ", predict(training_set,weak_classifer_list,alpha_list)
print "test error after 4 rounds: ", predict(test_set,weak_classifer_list,alpha_list)

train error after 4 rounds:  0.0511111111111
test error after 4 rounds:  0.0387596899225


# training error and test error output

In [14]:
weak_classifer_list,alpha_list =  boosting(training_set,3)
print "training error after 3 rounds: ", predict(training_set,weak_classifer_list,alpha_list)
print "test error after 3 rounds: ", predict(test_set,weak_classifer_list,alpha_list)

training error after 3 rounds:  0.0644444444444
test error after 3 rounds:  0.0387596899225


In [15]:
weak_classifer_list,alpha_list =  boosting(training_set,7)
print "training error after 7 rounds: ", predict(training_set,weak_classifer_list,alpha_list)
print "test error after 7 rounds: ", predict(test_set,weak_classifer_list,alpha_list)

training error after 7 rounds:  0.0288888888889
test error after 7 rounds:  0.031007751938


In [28]:
weak_classifer_list,alpha_list =  boosting(training_set,10)
print "training error after 10 rounds: ", predict(training_set,weak_classifer_list,alpha_list)
print "test error after 10 rounds: ", predict(test_set,weak_classifer_list,alpha_list)

training error after 10 rounds:  0.0155555555556
test error after 10 rounds:  0.0387596899225


In [29]:
weak_classifer_list,alpha_list =  boosting(training_set,15)
print "training error after 15 rounds: ", predict(training_set,weak_classifer_list,alpha_list)
print "test error after 15 rounds: ", predict(test_set,weak_classifer_list,alpha_list)

training error after 15 rounds:  0.0
test error after 15 rounds:  0.0232558139535


In [30]:
weak_classifer_list,alpha_list =  boosting(training_set,20)
print "training error after 20 rounds: ", predict(training_set,weak_classifer_list,alpha_list)
print "test error after 20 rounds: ", predict(test_set,weak_classifer_list,alpha_list)

training error after 20 rounds:  0.0
test error after 20 rounds:  0.0232558139535


notice that after15 rounds of boosting, the training err was reduced to 0,
which is caused by overfitting

# Based on the dictionary file, write down the words corresponding to the weak learners chosen in the first 10 rounds of boosting.

In [31]:
weak_classifer_list,alpha_list =  boosting(training_set,10)
print "training error after 7 rounds: ", predict(training_set,weak_classifer_list,alpha_list)
print "test error after 7 rounds: ", predict(test_set,weak_classifer_list,alpha_list)

training error after 7 rounds:  0.0155555555556
test error after 7 rounds:  0.0387596899225


In [32]:
print weak_classifer_list

[(3018, -1), (2001, 1), (1423, -1), (3783, 1), (2339, -1), (2089, 1), (620, -1), (1328, 1), (3899, -1), (886, 1)]


In [33]:
print "the words in dictionary are: "
for i in range(len(weak_classifer_list)):
    print dictionary[weak_classifer_list[i][0]]

the words in dictionary are: 
remove
language
free
university
money
linguistic
click
fax
want
de
